In [1]:
import transformers
import datasets
import random
import pandas as pd
import numpy as np
import nltk
import os
import bitsandbytes
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
from IPython.display import display, HTML
from datasets import load_dataset
from evaluate import load
from transformers.utils import send_example_telemetry
from bert_score import score

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, MT5ForConditionalGeneration
from transformers import (
    MBartForConditionalGeneration,
    MBart50Tokenizer,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
)
from sklearn.model_selection import KFold

from transformers import pipeline
import torch
print(transformers.__version__)
nltk.download('punkt')
nltk.download('punkt_tab')


4.45.1


[nltk_data] Downloading package punkt to /home/student/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/student/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [2]:
# Set random seeds for reproducibility
def set_random_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_random_seed(42)  # You can change this seed value as needed

# Check if CUDA is available and use it; otherwise, fall back to CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [3]:
raw_datasets = load_dataset(
    "mdaffarudiyanto/GR_custom_dataset",
    'stratified_articles_below_1000',
    data_dir="/home/student/Documents/MDR/custom_liputan6_data",
    ignore_verifications=True
)

model_checkpoint = "facebook/mbart-large-50-many-to-many-mmt"
model = MBartForConditionalGeneration.from_pretrained(model_checkpoint, device_map = "auto")
# model.to(device)
tokenizer = MBart50Tokenizer.from_pretrained(model_checkpoint)
tokenizer.src_lang = "id_ID"
tokenizer.tgt_lang = "id_ID"

rouge_metric = load("rouge")

max_input_length = 512
max_target_length = 128
prefix = "summarize: "

/home/student/Documents/MDR/.venv/lib/python3.10/site-packages/datasets/load.py:2524: FutureWarning: 'ignore_verifications' was deprecated in favor of 'verification_mode' in version 2.9.1 and will be removed in 3.0.0.
You can remove this warning by passing 'verification_mode=no_checks' instead.
  warnings.warn(
/home/student/Documents/MDR/.venv/lib/python3.10/site-packages/datasets/load.py:1461: FutureWarning: The repository for mdaffarudiyanto/GR_custom_dataset contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mdaffarudiyanto/GR_custom_dataset
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
/home/student/Documents/MDR/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokeni

In [4]:
lora_config = LoraConfig(
    r=256,
    lora_alpha=256,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.01,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)

model = prepare_model_for_kbit_training(model)

model = get_peft_model(model, lora_config)

model.print_trainable_parameters()

trainable params: 37,748,736 || all params: 648,628,224 || trainable%: 5.8198


In [5]:
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["clean_article"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding="max_length")

    # Setup the tokenizer for targets
    labels = tokenizer(text_target=examples["clean_summary"], max_length=max_target_length, truncation=True, padding="max_length")
 
    model_inputs["labels"] = labels["input_ids"]

    model_inputs["labels"] = [
    [(label if label != tokenizer.pad_token_id else -100) for label in labels_example]
    for labels_example in model_inputs["labels"]
    ]
    return model_inputs

tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)



In [6]:
batch_size = 8
model_name = model_checkpoint.split("/")[-1]
output_dir = f"GR-{model_name}-LoRA-20"

args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    num_train_epochs=10,
    learning_rate=1e-4,
    per_device_train_batch_size=batch_size,
    weight_decay=0.01,
    evaluation_strategy="no",  # Disable evaluation
    save_strategy="no",        # Disable checkpoint saving
    predict_with_generate=False,
    push_to_hub=False,
    load_best_model_at_end=False,  # No best model to load
)

label_pad_token_id = -100

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8,
)


# data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="pt")

/home/student/Documents/MDR/.venv/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [7]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    rouge_result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {
        "rouge1": rouge_result["rouge1"]* 100,
        "rouge2": rouge_result["rouge2"]* 100,
        "rougeL": rouge_result["rougeL"]* 100,
        "rougeLsum": rouge_result["rougeLsum"]* 100
    }

    return {k: round(v, 4) for k, v in result.items()}

In [8]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    # eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
model.config.use_cache = False

trainer.train()

  0%|          | 0/214960 [00:00<?, ?it/s]

{'loss': 2.3337, 'grad_norm': 1.163072109222412, 'learning_rate': 9.97673985857834e-05, 'epoch': 0.02}
{'loss': 2.172, 'grad_norm': 1.24675714969635, 'learning_rate': 9.953479717156681e-05, 'epoch': 0.05}
{'loss': 2.0427, 'grad_norm': 1.133284330368042, 'learning_rate': 9.930219575735021e-05, 'epoch': 0.07}
{'loss': 1.9979, 'grad_norm': 0.7249625325202942, 'learning_rate': 9.906959434313361e-05, 'epoch': 0.09}
{'loss': 2.0004, 'grad_norm': 1.1807817220687866, 'learning_rate': 9.883699292891701e-05, 'epoch': 0.12}
{'loss': 1.9617, 'grad_norm': 1.1896188259124756, 'learning_rate': 9.860439151470041e-05, 'epoch': 0.14}
{'loss': 1.9288, 'grad_norm': 1.2583374977111816, 'learning_rate': 9.837179010048381e-05, 'epoch': 0.16}
{'loss': 1.9122, 'grad_norm': 0.971587061882019, 'learning_rate': 9.813918868626722e-05, 'epoch': 0.19}
{'loss': 1.8708, 'grad_norm': 1.0308610200881958, 'learning_rate': 9.790658727205062e-05, 'epoch': 0.21}
{'loss': 1.8941, 'grad_norm': 1.0863723754882812, 'learning_ra

TrainOutput(global_step=214960, training_loss=1.3440924597265642, metrics={'train_runtime': 40856.7747, 'train_samples_per_second': 42.089, 'train_steps_per_second': 5.261, 'total_flos': 2.0627460559208448e+18, 'train_loss': 1.3440924597265642, 'epoch': 10.0})

In [9]:
# best_model_path = os.path.join(output_dir, "best_model")
# trainer.save_model(best_model_path)
# print(f"Best model saved to {best_model_path}")

In [10]:
peft_model_id = "mBART_LoRA_20_results"
trainer.model.save_pretrained(peft_model_id)
tokenizer.save_pretrained(peft_model_id)


('mBART_LoRA_20_results/tokenizer_config.json',
 'mBART_LoRA_20_results/special_tokens_map.json',
 'mBART_LoRA_20_results/sentencepiece.bpe.model',
 'mBART_LoRA_20_results/added_tokens.json')

GENERATE SUMMARIES

In [3]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
 

# Load peft config for pre-trained checkpoint etc.
peft_model_id = "mBART_LoRA_20_results"
config = PeftConfig.from_pretrained(peft_model_id)

# load base LLM model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path,  load_in_8bit=True,  device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.src_lang = "id_ID"
tokenizer.tgt_lang = "id_ID"
 
# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id, device_map={"":0})
model.eval()
 
print("Peft model loaded")


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
/home/student/Documents/MDR/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Peft model loaded


In [4]:
raw_datasets = load_dataset(
    "mdaffarudiyanto/GR_custom_dataset",
    'stratified_articles_below_1000',
    data_dir="/home/student/Documents/MDR/custom_liputan6_data",
    ignore_verifications=True
)

max_input_length = 512
max_target_length = 128
prefix = "summarize: "

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["clean_article"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding="max_length")

    # Setup the tokenizer for targets
    labels = tokenizer(text_target=examples["clean_summary"], max_length=max_target_length, truncation=True, padding="max_length")
 
    model_inputs["labels"] = labels["input_ids"]

    model_inputs["labels"] = [
    [(label if label != tokenizer.pad_token_id else -100) for label in labels_example]
    for labels_example in model_inputs["labels"]
    ]
    return model_inputs

tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)



/home/student/Documents/MDR/.venv/lib/python3.10/site-packages/datasets/load.py:2524: FutureWarning: 'ignore_verifications' was deprecated in favor of 'verification_mode' in version 2.9.1 and will be removed in 3.0.0.
You can remove this warning by passing 'verification_mode=no_checks' instead.
  warnings.warn(
/home/student/Documents/MDR/.venv/lib/python3.10/site-packages/datasets/load.py:1461: FutureWarning: The repository for mdaffarudiyanto/GR_custom_dataset contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mdaffarudiyanto/GR_custom_dataset
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [5]:
from torch.utils.data import DataLoader

# Define a custom collate function for DataLoader
def collate_fn(batch):
    articles = [item['clean_article'] for item in batch]
    summaries = [item['clean_summary'] for item in batch]

    # Tokenize the inputs
    inputs = tokenizer(
        articles,
        return_tensors='pt',
        truncation=True,
        padding='max_length',
        max_length=max_input_length
    )

    # Move tensors to the GPU
    inputs = {k: v.to(device) for k, v in inputs.items()}

    return inputs, articles, summaries

# Create the DataLoader
batch_size = 64  # Adjust based on your GPU memory
test_dataloader = DataLoader(
    raw_datasets["test"],
    batch_size=batch_size,
    collate_fn=collate_fn,
    shuffle=False  # Do not shuffle test data
)

In [6]:
import json  # Import json module at the top
from tqdm import tqdm 

# Function to generate and compare summaries
def generate_and_compare_summaries(model, tokenizer, dataloader, device):
    model.eval()

    display_data = []
    all_generated_summaries = []
    all_reference_summaries = []

    for inputs, articles, summaries in tqdm(dataloader, desc="Processing"):
        with torch.no_grad():
            summary_ids = model.generate(
                input_ids=inputs['input_ids'],
                attention_mask=inputs['attention_mask'],
                max_length=max_target_length,
                num_beams=5,
                early_stopping=True
            )

        # Decode the generated summaries
        generated_summaries = tokenizer.batch_decode(
            summary_ids,
            skip_special_tokens=True,
            clean_up_tokenization_spaces=True
        )

        # Store results
        for article, reference_summary, generated_summary in zip(articles, summaries, generated_summaries):
            display_data.append({
                "Original Text": article,
                "Reference Summary": reference_summary,
                "Generated Summary": generated_summary
            })
            all_generated_summaries.append(generated_summary)
            all_reference_summaries.append(reference_summary)

    # Calculate BERTScore
    P, R, F1 = score(
        all_generated_summaries,
        all_reference_summaries,
        model_type="bert-base-multilingual-cased",
        num_layers=9,
        lang='id',
        verbose=True
    )

    avg_precision = P.mean().item() * 100
    avg_recall = R.mean().item() * 100
    avg_f1 = F1.mean().item() * 100

    print(f"Average Precision: {avg_precision:.2f}")
    print(f"Average Recall: {avg_recall:.2f}")
    print(f"Average F1 Score: {avg_f1:.2f}")

    # Calculate ROUGE scores
    rouge_metric = load("rouge")
    rouge_scores = rouge_metric.compute(
        predictions=all_generated_summaries,
        references=all_reference_summaries,
        use_stemmer=True
    )
    
    rouge1 = rouge_scores['rouge1'] * 100
    rouge2 = rouge_scores['rouge2'] * 100
    rougeL = rouge_scores['rougeL'] * 100
    rougeLsum = rouge_scores['rougeLsum'] * 100

    print(f"ROUGE Scores:")
    print(f"ROUGE-1 F1 Score: {rouge1:.2f}")
    print(f"ROUGE-2 F1 Score: {rouge2:.2f}")
    print(f"ROUGE-L F1 Score: {rougeL:.2f}")
    print(f"ROUGE-Lsum F1 Score: {rougeLsum:.2f}")
    
    # Save ROUGE and BERTScore metrics to a JSON file
    scores_dict = {
        'BERTScore': {
            'Precision': avg_precision,
            'Recall': avg_recall,
            'F1': avg_f1
        },
        'ROUGE': {
            'ROUGE-1': rouge1,
            'ROUGE-2': rouge2,
            'ROUGE-L': rougeL,
            'ROUGE-Lsum': rougeLsum
        }
    }

    with open('evaluation_scores.json', 'w') as f:
        json.dump(scores_dict, f, indent=4)

    return pd.DataFrame(display_data)

# Run the summarization and evaluation
summary_comparison_df = generate_and_compare_summaries(
    model,
    tokenizer,
    test_dataloader,
    device
)

summary_comparison_df.to_csv("LoRA_summary_results.csv", index=False)

# Display the results in a DataFrame
display(HTML(summary_comparison_df.head(30).to_html(escape=False)))


Processing: 100%|██████████| 336/336 [26:59<00:00,  4.82s/it]


calculating scores...
computing bert embedding.


  0%|          | 0/665 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/336 [00:00<?, ?it/s]

done in 29.83 seconds, 720.85 sentences/sec
Average Precision: 79.59
Average Recall: 79.68
Average F1 Score: 79.60
ROUGE Scores:
ROUGE-1 F1 Score: 46.78
ROUGE-2 F1 Score: 31.47
ROUGE-L F1 Score: 40.93
ROUGE-Lsum F1 Score: 40.94


,Original Text,Reference Summary,Generated Summary
0,"Liputan6.com, Jakarta: Jalur tol Cikampek atau pun ruas tol arah Bogor masih tergolong rawan kejahatan. Banyak perampok beraksi dalam angkutan umum atau membegal sopir kendaraan niaga. Baru-baru ini, polisi berhasil menggagalkan perampokan truk yang tengah menuju Surabaya. Para perampok melaksanakan kejahatannya dengan dengan modus pura-pura menumpang. Tiga kawanan penjahat kelompok Sigit Haryanto adalah Sigit, Ikbal, dan Arip. Awalnya, mereka meminta izin menumpang saat truk tersebut mengisi bensin di daerah Cikampek. Mereka kemudian mendapat tempat di belakang truk. Namun, diam-diam, seorang rekan mereka, yakni Dedi Susanto membuntuti truk tersebut dengan mengendarai Toyota Kijang dengan plat nomor L-1355-ZX. Di tengah perjalanan, tiga perampok ini lalu menyergap awak truk. Mereka mengikat dan membekap mulut sopir dan kenek truk dan merampas kendaraan mereka. Kawanan penjahat itu kemudian memutar arah kembali ke Jakarta. Namun, tindakan mereka ternyata menyebabkan patroli polisi setempat merasa curiga. Polisi kemudian mengejar bandit-bandit dan berhasil menangkap mereka. Dua hari sebelumnya, petugas Kepolisian Resor Ciracas juga membekuk kawanan penjahat yang kerap beraksi di jalan tol Jakarta Timur. Kawanan itu sebelumnya menyamar sebagai penumpang angkutan umum. Mereka kemudian merampas harta penumpang di saat kendaraan itu melaju di jalan tol. Mereka juga mengancam korban dengan senjata tajam sehingga tak ada yang berani melawan. (HFS/Roy Achmad dan Kurnia Supriatna).","Perampokan kendaraan di jalan tol kian memprihatinkan. Selain merampok angkutan umum, perampok juga menyerang kendaraan niaga.","Tiga kawanan penjahat kelompok Sigit Haryanto adalah Sigit, Ikbal, dan Arip. Awalnya, mereka meminta izin menumpang saat truk tersebut mengisi bensin di daerah Cikampek."
1,"Liputan6.com, Semarang: Kepolisian Kota Besar Semarang berhasil menangkap sembilan tersangka bandar judi kupon putih. Bersamaan dengan itu, polisi juga menyita barang bukti berupa ratusan lembar kupon putih dan uang tunai jutaan rupiah. Sayangnya, bandar utama praktek perjudian ini belum berhasil ditangkap. Aparat Poltabes Semarang, saat ini tengah memeriksa secara intensif kesembilan tersangka, yang mau tak mau, harus mendekam di tahanan. Namun, keterangan mengenai sosok bandar dan beking di balik praktek judi ini, belum terungkap. Tersangka masih bungkam. Akhir-akhir ini, praktek perjudian memang tengah membanjiri Semarang. Tak pelak, keberadaan praktek perjudian ini cukup meresahkan masyarakat. Itu sebabnya, Komisi E DPRD Jawa Tengah meminta polisi lebih serius untuk memberantasnya. Masyarakat dan anggota dewan juga berharap, aparat Poltabes Semarang lebih sering lagi melakukan operasi represif ke berbagai lokasi penjualan kupon putih. Khususnya yang ada di daerah Rinjani, Plamongan, Jalan Badak dan Jalan Elisabeth. Selain mengamankan sembilan tersangka bandar kupon putih, Reserse Poltabes Semarang juga mengamankan dua tersangka judi koprok. Keduanya tertangkap ketika sedang menggelar perjudian di lapangan Simpang Lima Semarang. (ULF/Teguh Hadi Prayitno dan Kukuh Ary Wibowo).","Semarang mulai dibanjiri praktek perjudian. Namun, polisi gagal menangkap bandar utamanya.",Poltabes Semarang menangkap sembilan tersangka bandar judi kupon putih. Polisi juga menyita barang bukti berupa ratusan lembar kupon putih dan jutaan rupiah.
2,"Liputan6.com, Jakarta: Boleh jadi, Euro 2000 punyak kesan tersendiri bagi Pemerintah Daerah DKI Jakarta. Buktinya, olahraga akbar itu dijadikan jurus untuk meredam tawuran pelajar, yang semakin memprihatinkan itu. Tingginya angka perkelahian pelajar di Jakarta memang cukup memusingkan kepala. Setidaknya, bagi para pejabat Pemda DKI. Hampir tiap hari ada saja sekelompok pelajar yang bentrok dengan pelajar sekolah lainnya. Bagai seorang Gladiator, pelajar ini tidak sekedar berkelahi, tapi ada juga yang sengaja membunuh pelajar lainnya. Nah, untuk mengurangi angka tawuran pelajar ters